# Imports & Definitions

In [1]:
from SVD_utils import *
from Error_utils import *
import json
import numpy as np

# Creating arrayD

In [5]:
deleteArrays(D=True)
createDB("arrayD", "Temperatures", (num_images, image_height, image_width), (tile_thickness, tile_height, tile_width))
max_val, min_val, data_avg = populateArrayD()
vRange = max_val - min_val

param_dict = {}
parameter_names = ["max_val", "min_val", "data_avg", "vRange"]

for i in parameter_names:
    param_dict[i] = float(globals()[i])

with open("parameters.json", "w") as f:
    json.dump(param_dict, f)

# SVD related work

In [2]:
with open("parameters.json", "r") as f:
    param_dict = json.load(f)

for key, value in param_dict.items():
    globals()[key] = np.float32(value)

In [3]:
deleteArrays(D=False)
createDB("arrayUr", "U", db_shape=(num_SVD_matrices, SVD_shape[0], k), tile_shape=(1,SVD_shape[0],k), data_type=np.float32)
createDB("arraySr", "S", db_shape=(num_SVD_matrices, k, 1), tile_shape=(1,k,1), data_type=np.float32)
createDB("arrayVr", "V", db_shape=(num_SVD_matrices, k, SVD_shape[1]), tile_shape=(1,k,SVD_shape[1]), data_type=np.float32)

In [4]:
max_error, min_error, percent_error = compressData(data_avg, vRange)

100%|██████████| 1215/1215 [04:35<00:00,  4.42it/s]


In [5]:
# check quantizability
for i in [2**8, 2**16]:
    step = (max_error - min_error) / i
    do_quant = i if (step/2) / vRange < eps else 0
    
    if do_quant:
        break
    
print(do_quant)

E_datatype = np.float32

if do_quant == 2**8:
    E_datatype = np.uint8
elif do_quant == 2**16:
    E_datatype = np.uint16

256


In [6]:
step = (max_error - min_error) / 256
print((step/2) / vRange)

8.962008103625233e-05


In [7]:
print(percent_error)

#decide whether to be sparse here:
is_sparse = True if percent_error < 0.2 else False

0.129923545111063


In [8]:
is_sparse = True

In [9]:
createDB("arrayE", "E", (num_SVD_matrices, SVD_shape[0], SVD_shape[1]), (1, SVD_shape[0], SVD_shape[1]), sparse=is_sparse, data_type=E_datatype)
populateErrorArray(max_error, min_error, vRange, data_avg, do_quant, E_datatype, is_sparse)

100%|██████████| 1215/1215 [09:04<00:00,  2.23it/s]


In [12]:
param_dict = {}
parameter_names = ["max_val", "min_val", "data_avg", "vRange", "do_quant", "max_error", "min_error", "E_datatype"]

for i in parameter_names:
    if i == "E_datatype":
        param_dict[i] = str(globals()[i])

    else: param_dict[i] = float(globals()[i])

with open("parameters.json", "w") as f:
    json.dump(param_dict, f)

In [ ]:
with open("parameters.json", "r") as f:
    param_dict = json.load(f)

for key, value in param_dict.items():
    if key == "E_datatype":
        if "8" in value:
            
            E_datatype = np.uint8
        elif "16" in value:
            E_datatype = np.uint16
        else: E_datatype = np.float32
    
    globals()[key] = np.float32(value)

In [10]:
reconstructAndCheck(max_error, min_error, vRange, data_avg, do_quant, is_sparse)

TypeError: reconstructAndCheck() missing 1 required positional argument: 'is_sparse'

In [13]:
if getCompressionRatio() < 2:
    pass

16624609629
9459690128 1370337231
Compression ratio p: 1.5350477960874742
